**Zero shot classification**

In [14]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()  # Load environment variables from .env file, contains personal access token (HF_API_TOKEN=your_token)
headers = {"Authorization": f"Bearer {os.getenv('HF_API_TOKEN')}"}

candidate_labels = ["technology", "sports", "politics", "health"]

def query(model, input_text):
    API_URL = f"https://api-inference.huggingface.co/models/{model}"
    payload = {
        "inputs": input_text,
        "parameters": {"candidate_labels": candidate_labels}
    }
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

input_text = "I just bought a new laptop, and it works amazing!"

output = query("facebook/bart-large-mnli", input_text)
print(output)

{'sequence': 'I just bought a new laptop, and it works amazing!', 'labels': ['technology', 'health', 'sports', 'politics'], 'scores': [0.970917284488678, 0.014999152161180973, 0.008272469975054264, 0.005811101291328669]}


**Try to run a model locally**

In [35]:
# Prerequisites
from tabulate import tabulate
from transformers import pipeline
import json

# input text
input_text = "I like to cycle and I burn easily. I also love culture and like to post on social media about my food. I will go on a trip to italy in july."

# Load labels from a txt file
with open("labels.txt", "r", encoding="utf-8") as f:
    class_labels = [line.strip() for line in f if line.strip()]

# Load test data (in dictionary)
with open("test_data.json", "r") as file:
    packing_data = json.load(file)
# Get a list of trip descriptions (keys)
trips = list(packing_data.keys())
# Access the first trip description
first_trip = trips[0]
# Get the packing list for the secondfirst trip
first_trip_items = packing_data[first_trip]

print(f"First trip: {first_trip} \n")
print(f"Packing list: {first_trip_items}")

First trip: 7-Day Island Beach Holiday in Greece (Summer). I am planning a trip to Greece with my boyfriend, where we will visit two islands. We have booked an apartment on each island for a few days and plan to spend most of our time relaxing. Our main goals are to enjoy the beach, try delicious local food, and possibly go on a hike—if it’s not too hot. We will be relying solely on public transport. We’re in our late 20s and traveling from the Netherlands. 

Packing list: ['bathing suit', 'beach towel', 'beach bag', 'sandals', 'comfortable walking shoes', 'light jacket', 'sunscreen', 'sunglasses', 'sunhat', 'entertainment for downtime (e.g. book/ebook, games, laptop, journal)', 'short pants/skirts', 't-shirts/tops']


Load classifiers

In [36]:
# Load smaller the model and create a pipeline for zero-shot classification (1min loading + classifying with 89 labels)
classifier_bart_base = pipeline("zero-shot-classification", model="facebook/bart-base")

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


In [37]:
# Load larger the model and create a pipeline for zero-shot classification (5min loading model + classifying with 89 labels)
classifier_bart_large_mnli = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Try classifiers

In [38]:
# Run the classification (ca 30 seconds classifying)
result_bart_base = classifier_bart_base(first_trip, class_labels)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [39]:
# Run the classification (ca 1 minute classifying)
result_bart_large_mnli = classifier_bart_large_mnli(first_trip, class_labels)

In [40]:
# Creating a table
table = zip(result_bart_base["labels"], 
            result_bart_large_mnli["labels"])
headers = ["bart_base", "bart_large_mnli"]

print(tabulate(table, headers=headers, tablefmt="grid"))


+----------------------------------------------------------------------+----------------------------------------------------------------------+
| bart_base                                                            | bart_large_mnli                                                      |
+======================================================================+======================================================================+
| bandana                                                              | travel adapter                                                       |
+----------------------------------------------------------------------+----------------------------------------------------------------------+
| lip balm                                                             | travel journal                                                       |
+----------------------------------------------------------------------+----------------------------------------------------------------

**Try simple prompt engineering**

In [12]:
# No prompt
no_prompt = input_text
no_result = classifier(no_prompt, class_labels)

# Simple prompt
simple_prompt = "Classify the following text: " + input_text
simple_result = classifier(simple_prompt, class_labels)

# Primed prompt
primed_prompt = input_text + "What are the most important things to pack for the trip?"
primed_result = classifier(primed_prompt, class_labels)

In [13]:
# Creating a table
table = zip(no_result["labels"], no_result["scores"], 
            simple_result["labels"], simple_result["scores"], 
            primed_result["labels"], primed_result["scores"])
headers = ["no_prompt", "no_prompt", "simple_prompt", "simple_prompt", "primed_prompt", "primed_prompt"]

print(tabulate(table, headers=headers, tablefmt="grid"))


+----------------------------+-------------+----------------------------+-----------------+----------------------------+-----------------+
| no_prompt                  |   no_prompt | simple_prompt              |   simple_prompt | primed_prompt              |   primed_prompt |
+============================+=============+============================+=================+============================+=================+
| Travel-sized toiletries    |  0.0141621  | Beanie                     |      0.0126489  | First aid kit              |      0.0126422  |
+----------------------------+-------------+----------------------------+-----------------+----------------------------+-----------------+
| Refillable water bottle    |  0.013635   | Baby wipes                 |      0.0125994  | Work ID badge              |      0.0125781  |
+----------------------------+-------------+----------------------------+-----------------+----------------------------+-----------------+
| Aloe vera gel            